# Regime Model Comparison

## Objective
Compare three regime classification approaches:

1. **Multi-Factor Model (New)**: HMM using Returns + NFCI + Stock-Bond Correlation
2. **Binary Model (Baseline 1 - Intuition)**: Rule-based using Trend + VIX + Yield Spread
3. **Credit Spread Model (Baseline 2 - Old HMM)**: HMM using Credit Spread (BAA10Y)

## Key Metrics
- **Mismatch Ratio**: How often models disagree
- **Performance by Regime**: Sharpe Ratio and Returns for each regime

In [ ]:
# --- Imports ---
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt
from tqdm import tqdm

print("Libraries imported.")

## 1. Data Fetching

In [ ]:
def fetch_all_data(start_date='2002-07-01', end_date='2024-12-31'):
    """Fetch all data needed for the three models."""
    print(">>> Fetching Data...")
    
    # --- A. Market Data (SPY & TLT) ---
    spy = yf.download('SPY', start=start_date, end=end_date, interval='1mo', progress=False, auto_adjust=True)['Close']
    tlt = yf.download('TLT', start=start_date, end=end_date, interval='1mo', progress=False, auto_adjust=True)['Close']
    vix = yf.download('^VIX', start=start_date, end=end_date, interval='1mo', progress=False, auto_adjust=True)['Close']
    
    # Handle potential DataFrame format
    if isinstance(spy, pd.DataFrame): spy = spy.iloc[:, 0]
    if isinstance(tlt, pd.DataFrame): tlt = tlt.iloc[:, 0]
    if isinstance(vix, pd.DataFrame): vix = vix.iloc[:, 0]
    
    # Align indices to month-end
    spy.index = spy.index.to_period('M').to_timestamp('M')
    tlt.index = tlt.index.to_period('M').to_timestamp('M')
    vix.index = vix.index.to_period('M').to_timestamp('M')
    
    # --- B. Feature Engineering ---
    spy_returns = np.log(spy / spy.shift(1))
    tlt_returns = np.log(tlt / tlt.shift(1))
    correlation = spy_returns.rolling(window=12).corr(tlt_returns)
    spy_ma10 = spy.rolling(window=10).mean()
    
    # --- C. Macro Data from FRED ---
    nfci = web.DataReader('NFCI', 'fred', start_date, end_date).resample('M').last()
    credit_spread = web.DataReader('BAA10Y', 'fred', start_date, end_date).resample('M').last()
    yield_spread = web.DataReader('T10Y2Y', 'fred', start_date, end_date).resample('M').last()  # 10Y-2Y Spread
    
    nfci.index = nfci.index.to_period('M').to_timestamp('M')
    credit_spread.index = credit_spread.index.to_period('M').to_timestamp('M')
    yield_spread.index = yield_spread.index.to_period('M').to_timestamp('M')
    
    # --- D. Merge All Data ---
    df = pd.DataFrame({
        'SPY': spy,
        'TLT': tlt,
        'VIX': vix,
        'Returns': spy_returns,
        'Correlation': correlation,
        'MA10': spy_ma10
    })
    
    df = df.join(nfci).join(credit_spread).join(yield_spread).ffill().dropna()
    
    print(f"    Data Shape: {df.shape}")
    print(f"    Date Range: {df.index[0].strftime('%Y-%m')} to {df.index[-1].strftime('%Y-%m')}")
    
    return df

df = fetch_all_data()

## 2. Model Definitions

### 2.1 Multi-Factor Model (New)
HMM with Returns, NFCI, and Stock-Bond Correlation.

In [ ]:
def run_multi_factor_model(df, n_states=3, min_window=60):
    """Multi-Factor HMM: Returns + NFCI + Correlation."""
    print(">>> Running Multi-Factor Model...")
    
    regime_preds = []
    feature_cols = ['Returns', 'NFCI', 'Correlation']
    
    for t in tqdm(range(min_window, len(df))):
        train_data = df.iloc[:t + 1].copy()
        X_train = train_data[feature_cols].values
        
        means = X_train.mean(axis=0)
        stds = X_train.std(axis=0) + 1e-6
        X_scaled = (X_train - means) / stds
        
        best_score = -np.inf
        best_model = None
        
        for _ in range(5):
            try:
                model = GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=1000, tol=1e-4, min_covar=1e-3, random_state=None)
                model.fit(X_scaled)
                score = model.score(X_scaled)
                if score > best_score:
                    best_score = score
                    best_model = model
            except:
                continue
        
        if best_model is None:
            best_model = model
        
        hidden_states = best_model.predict(X_scaled)
        current_state = hidden_states[-1]
        
        # Dynamic State Re-ordering
        state_scores = []
        w_nfci, w_corr, w_ret = 0.5, 0.3, 0.2
        for i in range(n_states):
            m = best_model.means_[i]
            score = w_nfci * m[1] + w_corr * m[2] - w_ret * m[0]
            state_scores.append((i, score))
        
        sorted_states = sorted(state_scores, key=lambda x: x[1])
        mapping = {old: new for new, (old, _) in enumerate(sorted_states)}
        final_regime = mapping[current_state]
        regime_preds.append(final_regime)
    
    result_index = df.index[min_window:]
    return pd.Series(regime_preds, index=result_index, name='MultiFactorRegime')

### 2.2 Binary Model (Baseline 1 - Intuition)
Simple rule-based model: Bull if Price > MA10. Bear if Price < MA10 AND (VIX > Threshold OR Yield Spread < 0).

In [ ]:
def run_binary_model(df, vix_threshold=25):
    """Binary Rule-Based Model: Trend + VIX + Yield Spread."""
    print(">>> Running Binary Model...")
    
    regime = []
    for i in range(len(df)):
        price = df['SPY'].iloc[i]
        ma10 = df['MA10'].iloc[i]
        vix = df['VIX'].iloc[i]
        yield_spread = df['T10Y2Y'].iloc[i]
        
        if pd.isna(ma10) or pd.isna(vix) or pd.isna(yield_spread):
            regime.append(np.nan)
        elif price > ma10:
            regime.append(0)  # Bull
        elif vix > vix_threshold or yield_spread < 0:
            regime.append(2)  # Crisis/Bear
        else:
            regime.append(1)  # Uncertain
    
    return pd.Series(regime, index=df.index, name='BinaryRegime')

### 2.3 Credit Spread Model (Baseline 2 - Old HMM)
HMM with Credit Spread (BAA10Y) and Returns.

In [ ]:
def run_credit_spread_model(df, n_states=3, min_window=60):
    """Credit Spread HMM: BAA10Y + Returns."""
    print(">>> Running Credit Spread Model...")
    
    regime_preds = []
    feature_cols = ['Returns', 'BAA10Y']
    
    for t in tqdm(range(min_window, len(df))):
        train_data = df.iloc[:t + 1].copy()
        X_train = train_data[feature_cols].values
        
        means = X_train.mean(axis=0)
        stds = X_train.std(axis=0) + 1e-6
        X_scaled = (X_train - means) / stds
        
        best_score = -np.inf
        best_model = None
        
        for _ in range(5):
            try:
                model = GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=1000, tol=1e-4, min_covar=1e-3, random_state=None)
                model.fit(X_scaled)
                score = model.score(X_scaled)
                if score > best_score:
                    best_score = score
                    best_model = model
            except:
                continue
        
        if best_model is None:
            best_model = model
        
        hidden_states = best_model.predict(X_scaled)
        current_state = hidden_states[-1]
        
        # Ordering by credit spread (higher = crisis)
        state_scores = [(i, best_model.means_[i][1]) for i in range(n_states)]
        sorted_states = sorted(state_scores, key=lambda x: x[1])
        mapping = {old: new for new, (old, _) in enumerate(sorted_states)}
        final_regime = mapping[current_state]
        regime_preds.append(final_regime)
    
    result_index = df.index[min_window:]
    return pd.Series(regime_preds, index=result_index, name='CreditSpreadRegime')

## 3. Run All Models

In [ ]:
# Run models
multi_factor_regime = run_multi_factor_model(df)
binary_regime = run_binary_model(df)
credit_spread_regime = run_credit_spread_model(df)

# Combine results
results = df.copy()
results['MultiFactorRegime'] = multi_factor_regime
results['BinaryRegime'] = binary_regime
results['CreditSpreadRegime'] = credit_spread_regime
results = results.dropna()

print(f"\nResults Shape: {results.shape}")

## 4. Comparison Metrics

### 4.1 Mismatch Ratio

In [ ]:
def calculate_mismatch(regime1, regime2, name1, name2):
    """Calculate mismatch ratio between two regime series."""
    common_idx = regime1.dropna().index.intersection(regime2.dropna().index)
    r1 = regime1.loc[common_idx]
    r2 = regime2.loc[common_idx]
    
    # Compare Crisis detection (Regime 2)
    crisis_mismatch = (r1 == 2) != (r2 == 2)
    mismatch_ratio = crisis_mismatch.mean()
    
    print(f"Mismatch Ratio ({name1} vs {name2}): {mismatch_ratio:.2%}")
    return mismatch_ratio

print("=== Mismatch Analysis ===")
mismatch_mf_binary = calculate_mismatch(results['MultiFactorRegime'], results['BinaryRegime'], 'Multi-Factor', 'Binary')
mismatch_mf_credit = calculate_mismatch(results['MultiFactorRegime'], results['CreditSpreadRegime'], 'Multi-Factor', 'Credit Spread')
mismatch_binary_credit = calculate_mismatch(results['BinaryRegime'], results['CreditSpreadRegime'], 'Binary', 'Credit Spread')

### 4.2 Performance by Regime

In [ ]:
def calculate_regime_performance(df, regime_col, returns_col='Returns'):
    """Calculate performance metrics for each regime."""
    regime_data = df.dropna(subset=[regime_col])
    
    performance = []
    for regime in sorted(regime_data[regime_col].unique()):
        mask = regime_data[regime_col] == regime
        returns = regime_data.loc[mask, returns_col]
        
        if len(returns) > 0:
            ann_return = returns.mean() * 12
            ann_vol = returns.std() * np.sqrt(12)
            sharpe = ann_return / ann_vol if ann_vol > 0 else 0
            count = len(returns)
            
            regime_name = ['Bull', 'Uncertain', 'Crisis'][int(regime)]
            performance.append({
                'Regime': regime_name,
                'Count': count,
                'Annualized Return': f"{ann_return:.2%}",
                'Annualized Volatility': f"{ann_vol:.2%}",
                'Sharpe Ratio': f"{sharpe:.2f}"
            })
    
    return pd.DataFrame(performance)

print("\n=== Multi-Factor Model Performance ===")
display(calculate_regime_performance(results, 'MultiFactorRegime'))

print("\n=== Binary Model Performance ===")
display(calculate_regime_performance(results, 'BinaryRegime'))

print("\n=== Credit Spread Model Performance ===")
display(calculate_regime_performance(results, 'CreditSpreadRegime'))

## 5. Visualization

In [ ]:
def plot_regime_comparison(df, regime_col, title):
    """Plot SPY price with regime coloring."""
    fig, ax = plt.subplots(figsize=(14, 4))
    
    colors = ['#2ca02c', '#ff7f0e', '#d62728']  # Green, Orange, Red
    labels = ['Bull', 'Uncertain', 'Crisis']
    
    regime_data = df.dropna(subset=[regime_col])
    
    for i in range(3):
        mask = regime_data[regime_col] == i
        ax.scatter(
            regime_data.index[mask],
            regime_data['SPY'][mask],
            s=15, c=colors[i], label=labels[i], alpha=0.7
        )
    
    ax.set_title(title)
    ax.set_yscale('log')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_ylabel('SPY Price (Log)')
    plt.tight_layout()
    plt.show()

plot_regime_comparison(results, 'MultiFactorRegime', 'Multi-Factor Model (Returns + NFCI + Correlation)')
plot_regime_comparison(results, 'BinaryRegime', 'Binary Model (Trend + VIX + Yield Spread)')
plot_regime_comparison(results, 'CreditSpreadRegime', 'Credit Spread Model (BAA10Y + Returns)')

## 6. Hidden Risk Analysis
Identify cases where the Binary Model (Intuition) predicts Bull, but the Multi-Factor Model flags Crisis.

In [ ]:
hidden_risk = results[(results['BinaryRegime'] == 0) & (results['MultiFactorRegime'] == 2)]

print(f"=== Hidden Risk Cases ===")
print(f"Number of months where Binary=Bull but Multi-Factor=Crisis: {len(hidden_risk)}")

if len(hidden_risk) > 0:
    print(f"\nAverage 'Next Month' Return in these cases: {hidden_risk['Returns'].shift(-1).mean():.2%}")
    print(f"\nDates:")
    for date in hidden_risk.index:
        print(f"  - {date.strftime('%Y-%m')}")

## 7. Summary Table

In [ ]:
summary = pd.DataFrame({
    'Model': ['Multi-Factor', 'Binary', 'Credit Spread'],
    'Type': ['HMM (3 Features)', 'Rule-Based', 'HMM (2 Features)'],
    'Features': [
        'Returns, NFCI, Correlation',
        'Trend, VIX, Yield Spread',
        'Returns, Credit Spread'
    ],
    'Mismatch vs Binary': [f"{mismatch_mf_binary:.2%}", '-', f"{mismatch_binary_credit:.2%}"],
    'Mismatch vs Credit Spread': [f"{mismatch_mf_credit:.2%}", f"{mismatch_binary_credit:.2%}", '-']
})

print("=== Model Comparison Summary ===")
display(summary)

## 8. Save Results

In [ ]:
# Save regime data for further analysis
output_path = '../data/processed/regime_comparison.csv'
results[['SPY', 'Returns', 'MultiFactorRegime', 'BinaryRegime', 'CreditSpreadRegime']].to_csv(output_path)
print(f"Results saved to: {output_path}")